In [1]:
import matplotlib.pyplot as plt
import torch
import pytorch_measure as pm
N = 50 # number of weights
verbose = True
dev = 'cpu'

data = torch.randn(10000).to(dev)
#l = [min(data).item(), max(data).item()]
w = torch.softmax(torch.ones(N,dtype=torch.float),dim=-1).to(dev)
w = torch.nn.parameter.Parameter(w)
l = torch.linspace(-4, 4, N, requires_grad=False).to(dev)
# if l is min & max only
#index = [round((y.item() - l[0]) * (N - 1) / (l[1] - l[0])) for y in data]
# if l is all locations
index = torch.argmin(abs(l-data.view(-1,1)), dim=1)
def NLLLoss(m:list[pm.Measure]):
    mask = torch.zeros(len(m[0].weights), dtype=torch.bool)
    mask[m[0].support()] = 1
    return -m[0].weights[index].log()[mask[index]].sum()

sd = (data - l[index])**2
def WardLoss(w):
    return sum(sd * w[index])


In [3]:
measure = pm.Measure(locations=l, weights=w, device=dev)
opt = pm.Optimizer([measure], lr=1e-1)
opt.minimize(NLLLoss, verbose=True)

plt.bar(height=measure.weights.detach(),x=l)
plt.hist(torch.randn(10000),bins=40, density=True, range=[-4,4], alpha=0.5)
plt.legend(['Model','True data'])
plt.show()


Epoch: 0          Loss: 30411      LR: [0.1]
Epoch: 100        Loss: 26624      LR: [0.1]
Epoch: 200        Loss: 26624      LR: [0.1]
Epoch: 300        Loss: 26624      LR: [0.1]
Epoch: 400        Loss: 26624      LR: [0.1]
Epoch: 500        Loss: 26624      LR: [0.1]
Epoch: 600        Loss: 26624      LR: [0.1]
Epoch: 700        Loss: 26624      LR: [0.1]
Epoch: 800        Loss: 26624      LR: [0.1]
Epoch: 900        Loss: 26624      LR: [0.1]
Epoch: 1000       Loss: 26624      LR: [0.1]
Epoch: 1100       Loss: 26624      LR: [0.1]
Epoch: 1200       Loss: 26624      LR: [0.1]
Epoch: 1300       Loss: 26624      LR: [0.1]
Epoch: 1400       Loss: 26624      LR: [0.1]
Epoch: 1500       Loss: 26624      LR: [0.1]
Epoch: 1600       Loss: 26624      LR: [0.1]
Epoch: 1700       Loss: 26624      LR: [0.1]
Epoch: 1800       Loss: 26624      LR: [0.1]
Epoch: 1900       Loss: 26624      LR: [0.1]
Epoch: 2000       Loss: 26624      LR: [0.1]
Epoch: 2100       Loss: 26624      LR: [0.1]
Epoch: 220

KeyboardInterrupt: 